In [1]:
import cycpd
import matplotlib.pyplot as plt
import pycpd
import numpy as np
import os
import glob
import shutil
import open3d as o3d
import time
import h5py
import copy
import trimesh
from utils import rotationMatrixToEulerAngles, fixedNumDownSample, voxelDownSample, showPointCloud, surfaceVertices2WatertightO3dMesh, loadAlignedPointGroupsWithIndex, getEigValVecOfSSMByPCA

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [58]:
toothIndex = 37
loadDir = r"./data/cpdAlignedData/{}/".format(toothIndex)
alignedPointGroups, alignedPgTags = loadAlignedPointGroupsWithIndex(loadDir)

In [59]:
eigVal, eigVec, A, meanTrainPointVector = getEigValVecOfSSMByPCA(alignedPointGroups)

## TEST

In [60]:
priorMeanPG = meanTrainPointVector.reshape(-1,3).astype(np.double)
priorMeanMesh = surfaceVertices2WatertightO3dMesh(priorMeanPG)

In [66]:
testDataIndex = 17
testMeshPath = r"F:\Smartee\dataWithPhoto\format-obj\{}\{}L.obj".format(toothIndex,testDataIndex)
# testRepairedMeshPath = r"F:\Smartee\dataWithPhoto\repaired-obj\{}\{}U.obj".format(toothIndex,testDataIndex)

In [67]:
testMesh = o3d.io.read_triangle_mesh(testMeshPath)
# testRepairedMesh = o3d.io.read_triangle_mesh(testRepairedMeshPath)

In [68]:
# print("area ratio :{:.2f}/{:.2f}".format(testMesh.get_surface_area(), testRepairedMesh.get_surface_area()))

In [69]:
voxelSize = 0.2
v, f = trimesh.remesh.subdivide_to_size(vertices=np.asarray(testMesh.vertices), faces=np.asarray(testMesh.triangles), max_edge=voxelSize, max_iter=10, return_index=False)
# v = voxelDownSample(v, voxelSize)
v = fixedNumDownSample(v, desiredNumOfPoint=2500, leftVoxelSize=1.0, rightVoxelSize=0.1)
print("downsampled test pointcloud shape: ",v.shape)

downsampled test pointcloud shape:  (2500, 3)


In [70]:
# 显示网格化下采样后的存在缺损的测试点云
showPointCloud(v, windowName="re-sampled data with missing part")

In [71]:
X = np.array(v,dtype=np.double).copy() #参考点云：存在缺损的测试点云，但是牙龈牙齿分界线完整
Y = priorMeanPG.copy() #移动点云：先验平均点云
reg = cycpd.rigid_registration(**{'X': X, 'Y': Y,'max_iterations':300,'tolerance':1e-4,'verbose':True,'print_reg_params':True}) #缩放刚性配准确定缩放尺寸
TY,(s,r,t) = reg.register()

Iteration:1
ML: -2071.605; 	ML change (error):  2071.605; 	Sigma^2:     6.359; 	Sigma^2 change:    26.675
[                                                                        ]
Iteration:2
ML: -7425.424; 	ML change (error):  5353.818; 	Sigma^2:     6.102; 	Sigma^2 change:     0.257
[                                                                        ]
Iteration:3
ML: -7486.553; 	ML change (error):    61.129; 	Sigma^2:     5.793; 	Sigma^2 change:     0.309
[                                                                        ]
Iteration:4
ML: -7539.643; 	ML change (error):    53.090; 	Sigma^2:     5.366; 	Sigma^2 change:     0.427
[                                                                        ]
Iteration:5
ML: -7617.674; 	ML change (error):    78.031; 	Sigma^2:     4.818; 	Sigma^2 change:     0.548
[=                                                                       ]
Iteration:6
ML: -7726.800; 	ML change (error):   109.125; 	Sigma^2:     4.182; 	Sigma^2 change:

In [95]:
reg2 = cycpd.affine_registration(**{'X': X, 'Y': TY,'max_iterations':100,'tolerance':1e-4,'verbose':True,'print_reg_params':True}) #仿射变换配准，保证牙齿轮廓边缘相匹配
TTY,(_B,_t) = reg2.register()
# reg2 = cycpd.deformable_registration(**{'X': X, 'Y': TY,'max_iterations':100,'tolerance':1e-4,'verbose':True,'print_reg_params':True}) #自由变换配准，保证牙齿轮廓边缘相匹配
# TTY,(_G,_w) = reg2.register()

Iteration:1
ML: -6093.769; 	ML change (error):  6093.769; 	Sigma^2:     5.031; 	Sigma^2 change:     7.312
[                                                                        ]
Iteration:2
ML: -7856.945; 	ML change (error):  1763.176; 	Sigma^2:     3.778; 	Sigma^2 change:     1.254
[=                                                                       ]
Iteration:3
ML: -8281.972; 	ML change (error):   425.027; 	Sigma^2:     2.754; 	Sigma^2 change:     1.024
[==                                                                      ]
Iteration:4
ML: -8740.486; 	ML change (error):   458.514; 	Sigma^2:     2.013; 	Sigma^2 change:     0.741
[==                                                                      ]
Iteration:5
ML: -9126.655; 	ML change (error):   386.169; 	Sigma^2:     1.551; 	Sigma^2 change:     0.462
[===                                                                     ]
Iteration:6
ML: -9396.348; 	ML change (error):   269.693; 	Sigma^2:     1.243; 	Sigma^2 change:

In [96]:
def showRefAndMovPointClouds(pRef,pMov):
    # X: reference pointcloud, np.array
    # TY: transformed moving pointcloud, np.array
    pcdTY = o3d.geometry.PointCloud()
    pcdTY.points = o3d.utility.Vector3dVector(pMov)
    pcdTY.paint_uniform_color(np.array([1.,0.,0.]))
    pcdX = o3d.geometry.PointCloud()
    pcdX.points = o3d.utility.Vector3dVector(pRef)
    pcdX.paint_uniform_color(np.array([0.,0.,1.]))
    o3d.visualization.draw_geometries([pcdX,pcdTY], window_name="Reference PCL(blue) and Moving PCL(red)", width=800, height=600, left=50,top=50, point_show_normal=False)

In [97]:
def computeTX(X, s, R, t): #计算反向移动的测试点云
    return 1./s * (X - t) @ R.T

In [98]:
TX = computeTX(X, s, r, t)

In [99]:
def getCorrePointPairsWithMinProb(probabilityMatrix, minProb):
    """根据cpd的概率矩阵，获取对应点对"""
    """matProb.shape=(pMovNum,pRefNum)"""
    matProb = probabilityMatrix.copy()
    pointPairs = []
    pRefNum = matProb.shape[1]
    for i in range(pRefNum): #顺序贪婪（快）
        j = np.argmax(matProb[:,i])
        if matProb[j,i] > minProb:
            pointPairs.append((i, j)) # i:ref index, j: mov index
            matProb[j,:] = 0.0
    return np.array(pointPairs, dtype=np.uint32)

def extractCorreMovPoints(pMov, pointPairs):
    return pMov[pointPairs[:,1], :]

def extractCorreRefPoints(pRef, pointPairs):
    return pRef[pointPairs[:,0], :]

In [100]:
def findOptimalMinProb(matProb, priorPNum, priorMeanMesh, testBrokenMesh, scaleFactor):
    """找到最优的概率阈值使得 priorMeanMesh的选中区域的面积 * scaleFactor^2 == testBrokenMesh的面积"""
    leftProb = 1.0
    rightProb = 0.0
    midProb = (leftProb + rightProb) / 2.
    priorArea = priorMeanMesh.get_surface_area()
    obserArea = testBrokenMesh.get_surface_area()
    pNum = getCorrePointPairsWithMinProb(matProb, midProb).shape[0]
    expectedPNum = obserArea / (priorArea * scaleFactor**2) * priorPNum
    print("expected num of point in test broken pointcloud: ",expectedPNum)
    while np.abs(pNum - expectedPNum) > 1:
        if pNum < expectedPNum:
            leftProb = copy.copy(midProb)
        else:
            rightProb = copy.copy(midProb)
        midProb = (leftProb + rightProb) / 2.
        pNum = getCorrePointPairsWithMinProb(matProb, midProb).shape[0]
    return midProb

In [101]:
optimalInfProb = findOptimalMinProb(reg2.P, 1500, priorMeanMesh, testMesh, s)

expected num of point in test broken pointcloud:  1126.5541230830142


In [102]:
pointPairs = getCorrePointPairsWithMinProb(reg2.P, minProb=optimalInfProb)
pointPairsOrder = sorted(range(len(pointPairs)), key=lambda x:pointPairs[x,1]) # 按照先验平均点云的index进行排序
pointPairs = pointPairs[pointPairsOrder]

In [103]:
pointPairs.shape

(1127, 2)

In [104]:
pointPairs.shape[0]/priorMeanPG.shape[0] * priorMeanMesh.get_surface_area() * s**2

165.5984245735067

In [105]:
testMesh.get_surface_area() 

165.5329085885852

In [106]:
correY = extractCorreMovPoints(Y, pointPairs)

In [107]:
correTX = extractCorreRefPoints(TX, pointPairs)
correX = extractCorreRefPoints(X, pointPairs)

In [108]:
showRefAndMovPointClouds(Y, TX)

In [109]:
# showRefAndMovPointClouds(X,correX+0.01)

In [110]:
showRefAndMovPointClouds(correY,correTX) # correY:corresponding priorMeanPG, correTX: corresponding transformed testPG

In [154]:
# def reconstructMissingPoints(priorMeanVec, priorCovMat, obserVec, obserIndices):
#     # Get the predicted mean vec(predMeanVec), posterior mean vec(posteriorMuX) and covMat(posteriorCovMatXX) of a Gaussian process
#     # X:missing part, Y:observed data, This function works if covMat is non-singular
#     nonObserIndices = np.setdiff1d(np.arange(priorMeanVec.shape[0]), obserIndices, assume_unique=False)
#     assert np.abs(np.linalg.det(covMat))>1e-6, "Covariance matrix is singular."
#     muX = priorMeanVec[nonObserIndices]
#     muY = priorMeanVec[obserIndices]
#     covMatXY = covMat[nonObserIndices[:,None],obserIndices[None,:]]
#     covMatYY = covMat[obserIndices[:,None],obserIndices[None,:]]
#     covMatXX = covMat[nonObserIndices[:,None],nonObserIndices[None,:]]
#     invCovMatYY = np.linalg.inv(covMatYY) #可能不可逆！
#     posteriorMuX = muX + covMatXY @ invCovMatYY @ (obserVec-muY)
#     print(covMatXY @ invCovMatYY @ (obserVec-muY))
#     posteriorCovMatXX = covMatXX - covMatXY @ invCovMatYY @ covMatXY.T
#     predMeanVec = np.zeros(priorMeanVec.shape, priorMeanVec.dtype)
#     predMeanVec[nonObserIndices] = posteriorMuX
#     predMeanVec[obserIndices] = obserVec
#     return predMeanVec, posteriorMuX, posteriorCovMatXX

In [36]:
obserIndices = np.vstack([3*pointPairs[:,1], 3*pointPairs[:,1]+1, 3*pointPairs[:,1]+2]).T.flatten()
nonObserIndices = np.setdiff1d(np.arange(3*priorMeanPG.shape[0]), obserIndices, assume_unique=False)
print(obserIndices[:9])
print(pointPairs[:3,1])

[ 0  1  2 12 13 14 15 16 17]
[0 4 5]


In [37]:
# sampleNum = A.shape[0]
# covMat = 1./(sampleNum-1) * A.T @ A
# print("covMat is non-singular: ",np.linalg.det(covMat)>1e-6)
# reconsTestPG, reconsTestMissingPG, _ = reconstructMissingPoints(priorMeanVec=TY.flatten(), priorCovMat=covMat, obserVec=correX.flatten(), obserIndices=obserIndices)

In [38]:
# subV @ featureVec = observedVec - observedPriorVecMu 最小二乘解featureVec
numPC2Keep = 60
V = eigVec[:,:numPC2Keep]
subV = eigVec[obserIndices,:numPC2Keep]
subVc = eigVec[nonObserIndices,:numPC2Keep] 
lambd = 0.0 # a small constant to avoid over-fitting while maintaining acceptable residuals
featureVec = np.linalg.inv(subV.T @ subV + lambd*np.identity(numPC2Keep)) @ subV.T @ (correTX - correY).flatten() 

In [39]:
predTX = Y + (featureVec @ V.T).reshape(-1,3)
predX = s * predTX @ r + t

In [40]:
showRefAndMovPointClouds(X,predX) # X:testPG, predX:predicted resampled testPG

In [160]:
showRefAndMovPointClouds(Y,predTX) # Y:priorMeanPG, predTX:transformed predicted resampled testPG

In [161]:
assemX = predX.copy()
assemX[pointPairs[:,1]] = correX

In [162]:
showRefAndMovPointClouds(X,assemX)

In [163]:
showPointCloud(predX,"predX")